# 1.8

In [1]:
import numpy as np

def ra_to_degrees(ra_h, ra_m, ra_s):
    """
    Convert Right Ascension from H:M:S to decimal degrees.
    
    RA is typically given in hours, minutes, and seconds:
      24h = 360°
    => 1 hour = 15 degrees
    
    Parameters:
    ra_h : RA in hours
    ra_m : RA in minutes
    ra_s : RA in seconds
    
    Returns:
    RA in decimal degrees
    """
    return 15.0 * (ra_h + ra_m/60.0 + ra_s/3600.0)

def dec_to_degrees(dec_deg, dec_arcmin, dec_arcsec):
    """
    Convert Declination from D:M:S to decimal degrees.
    
    Dec is given in degrees, arcminutes, and arcseconds.
    If the declination is negative, the degrees are negative.
    
    Parameters:
    dec_deg    : degrees (can be negative)
    dec_arcmin : arcminutes
    dec_arcsec : arcseconds
    
    Returns:
    Declination in decimal degrees
    """
    # Keep track of the sign from the degree component
    sign = 1.0 if dec_deg >= 0 else -1.0
    # Convert arcminutes and arcseconds to degrees
    abs_dec = abs(dec_deg) + (dec_arcmin / 60.0) + (dec_arcsec / 3600.0)
    return sign * abs_dec

def degrees_to_ra(ra_deg):
    """
    Convert RA in decimal degrees to (hours, minutes, seconds).
    
    Parameters
    ----------
    ra_deg : float
        Right Ascension in decimal degrees, typically [0, 360).
    
    Returns
    -------
    (ra_h, ra_m, ra_s) : tuple of (int, int, float)
        RA in hours (0–23), minutes (0–59), and (float) seconds (0–59.999...).
    """
    # Ensure RA is within [0, 360)
    ra_deg = ra_deg % 360.0
    
    # Convert degrees to "RA hours"
    ra_hours = ra_deg / 15.0  # 360 deg = 24h => 1h = 15 deg
    
    # Extract H, M, S from the fractional hours
    ra_h = int(ra_hours)
    remainder = (ra_hours - ra_h) * 60.0
    ra_m = int(remainder)
    ra_s = (remainder - ra_m) * 60.0
    
    return ra_h, ra_m, ra_s

def degrees_to_dec(dec_deg):
    """
    Convert Declination in decimal degrees to (degrees, arcminutes, arcseconds).
    
    Parameters
    ----------
    dec_deg : float
        Declination in decimal degrees, can be negative (e.g., -26.5).
    
    Returns
    -------
    (dec_d, dec_m, dec_s) : tuple of (int, int, float)
        Declination as degrees, minutes, seconds.
        'dec_d' can be negative; 'dec_m' and 'dec_s' are typically non-negative.
    """
    # Capture the sign and work with absolute degrees
    sign = 1 if dec_deg >= 0 else -1
    abs_dec = abs(dec_deg)
    
    # Break down into D, M, S
    dec_d = int(abs_dec)
    remainder = (abs_dec - dec_d) * 60.0
    dec_m = int(remainder)
    dec_s = (remainder - dec_m) * 60.0
    
    # Reapply sign to the degrees portion only
    dec_d = sign * dec_d
    
    return dec_d, dec_m, dec_s

def convert_to_degrees(coords):
    RA = ra_to_degrees(coords[0][0], coords[0][1], coords[0][2])
    dec = dec_to_degrees(coords[1][0], coords[1][1], coords[1][2])
    return (RA,dec)

def convert_to_coords(degrees):
    ra_h, ra_m, ra_s = degrees_to_ra(degrees[0])
    dec_d, dec_m, dec_s = degrees_to_dec(degrees[1])
    return ((ra_h, ra_m, ra_s),(dec_d, dec_m, dec_s))

def calculate_angular_difference(A,B):
    '''
    Takes in two stars and emits the angular distance between them.
    
    Star A and B should be entered in (α,δ) format.
    '''
    A_α_δ = convert_to_degrees(A)
    B_α_δ = convert_to_degrees(B)
    Δα = A_α_δ[0] - B_α_δ[0]
    Δδ = A_α_δ[1] - B_α_δ[1]
    avg_δ = (A_α_δ[1] + B_α_δ[1])/2
    cos_δ = np.cos(np.radians(avg_δ))
    
    θ = ((Δα * cos_δ)**2 + (Δδ)**2)**.5
    
    return θ

### 1.8 a.

In [2]:
# Format = ((h,m,s),(°,',"))
P = ((14,29,42.95),(-62,40,46.1)) # Proxima Centauri
A = ((14,39,36.50),(-60,50,02.3)) # Alpha Centauri

P_d = convert_to_degrees(P)
A_d = convert_to_degrees(A)

P_d, A_d

((217.4289583333333, -62.67947222222222),
 (219.90208333333334, -60.83397222222222))

In [3]:
Δθ = calculate_angular_difference(P,A)
Δθ

2.185297782328779

### 1.8 b.

In [4]:
r = 4.0e16

d = r * np.sin(np.radians(Δθ)) # Distance between proxima and alpha centauri
d

1525255794320460.5

# 1.9

### 1.9 a.

In [5]:
t_j = 2000.0

t = 2010.0

T = (t - t_j)/100

# All coefficients in degrees

M = 1.2812323 * T + 0.0003879 * T**2 + 0.0000101 * T**3
N = 0.5567530 * T - 0.0001185 * T**2 - 0.0000116 * T**3

In [6]:
P_d

(217.4289583333333, -62.67947222222222)

In [7]:
α_deg = P_d[0]
δ_deg = P_d[1]

Δα_deg_pres = M + N * np.sin(α_deg) * np.tan(δ_deg)
Δδ_deg_pres = N * np.cos(α_deg)

α_deg, Δα_deg_pres, δ_deg, Δδ_deg_pres

(217.4289583333333,
 0.12289131904968786,
 -62.67947222222222,
 -0.04401399789156724)

In [8]:
new_coords = convert_to_coords((α_deg + Δα_deg_pres ,δ_deg + Δδ_deg_pres))
new_coords

((14, 30, 12.443916571919544), (-62, 43, 24.550392409649362))

### 1.9 b.

In [9]:
φ = 282 # position angle, in degrees

# Format =(°,',")
dθ = (0,0,3.84)
Δθ = (t - t_j) * np.array(dθ)
Δθ_d = dec_to_degrees(Δθ[0],Δθ[1], Δθ[2])

Δα_deg_pm = Δθ_d * np.sin(np.radians(φ))/np.cos(np.radians(δ_deg))
Δδ_deg_pm = Δθ_d * np.cos(np.radians(φ))

Δα_deg_pm, Δδ_deg_pm

(-0.0227326889068257, 0.0022177247020561007)

### 1.9 c.

In [10]:
print('Change in right ascension from precession = ',round(Δα_deg_pres,4),'; from proper motion = ', round(Δα_deg_pm,4))
print('Change in declination from precession = ',round(Δδ_deg_pres,4),'; from proper motion = ', round(Δδ_deg_pm,4))
print('\nThe change due to precession is much more signifigant, which makes sense - \nprcession happens on much shorter timescales.')

Change in right ascension from precession =  0.1229 ; from proper motion =  -0.0227
Change in declination from precession =  -0.044 ; from proper motion =  0.0022

The change due to precession is much more signifigant, which makes sense - 
prcession happens on much shorter timescales.
